# Lab 7: Model Selection, Regularization, and Cross-Validation

In this lab, you will practice using `scikit-learn` to generate models of various complexity. You'll then use the holdout method and K-fold cross-validation to select the models that generalize best.

**Due Date: Monday, November 25, 11:59 PM PT.**

### Collaboration Policy
Data science is a collaborative activity. While you may talk with others about this assignment, we ask that you **write your solutions individually**. If you discuss the assignment with others, please **include their names** in the cell below.

**Collaborators:** *list names here*

In [2]:
# Run this cell to set up your notebook
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline
sns.set()
sns.set_context("talk")

#from IPython.display import display, Latex, Markdown

### Introduction

For this lab, we will use a dataset to predict the house prices in Boston.

In [3]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [4]:
boston = pd.DataFrame(data)
boston.columns = ["CRIM", "ZN", "INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT"]
boston_target = target
boston.drop("B", axis = 1,inplace = True)
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,5.33


This dataset relates a number of features to the median house value of occupied homes. 

 - CRIM     per capita crime rate by town
 - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
 - INDUS    proportion of non-retail business acres per town
 - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
 - NOX      nitric oxides concentration (parts per 10 million)
 - RM       average number of rooms per dwelling
 - AGE      proportion of owner-occupied units built prior to 1940
 - DIS      weighted distances to five Boston employment centres
 - RAD      index of accessibility to radial highways
 - TAX      full-value property-tax rate per $10,000
 - PTRATIO  pupil-teacher ratio by town
 - LSTAT    % lower status of the population
 - Target:   Median value of owner-occupied homes in 1000s USD

The following cell will create a Dataframe of the features of the Boston data as well as a target variable representing the variable we wish to predict (housing value). 

#### Question 1

Let's model this housing price data! Before we can do this, however, we need to split the data into training and test sets. Remember that the response vector (housing prices) lives in the `target` attribute. A random seed is set here so that we can deterministically generate the same splitting in the future if we want to test our result again and find potential bugs.

Use the [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to split out 10% of the data for the test set. Call the resulting splits `X_train`, `X_holdout`, `Y_train`, `Y_holdout`. Here "holdout" refers to the fact that we're going to hold this data our when training our model.

In [5]:
from sklearn.model_selection import train_test_split
np.random.seed(45)

X = boston
Y = target

X_train, X_Holdout, Y_train, Y_holdout = train_test_split(X, Y, test_size = .1)

#### Question 2.1

As a warmup, fit a linear model to describe the relationship between the housing price and all available covariates. We've imported `sklearn.linear_model` as `lm`, so you can use that instead of typing out the whole module name. Fill in the cells below to 

1) fit a linear regression model to the covariates using the training data, then 

2) predict housing values using the holdout x values, and 

3) create a scatter plot for these predictions vs. the true prices

In [ ]:
len(X_train)

In [ ]:
import sklearn.linear_model as lm

linear_model = lm.LinearRegression()

# Fit your linear model
linear_model.fit(X_train,Y_train)

# Predict housing prices on the test set
Y_pred = linear_model.predict(X_Holdout)


In [ ]:
plt.scatter(Y_pred, Y_holdout)
plt.xlabel("predicted y_values"); plt.ylabel("Actual Y-values"); plt.title("Scatterplot of Predicted vs Actual Y-values")

#### Question 2.2

Briefly analyze the scatter plot above. Do you notice any outliers? Write your answer in the cell below.

**Your response**:  I wouldn't say that there are any distinct outliers considering we want things to be at the intersection between the axis ticks. For the most part everything increases one to one. Two "outliers" I guess would be at a predicted value of 10 and a 25 actual, while we also had a predicted value of 10 and had a 24 ish actual/ 

Alternatively, we can plot the residuals vs. our model predictions. Ideally they'd all be zero. Given the inevitably of noise, we'd at least like them to be scatter randomly across the line where the residual is zero. These residuals appear to be pretty good, though it seems evident that our housing model is overpredicting housing prices at low and high real values. Note that this might vary a little bit depending on your random sample of training data. 

In [ ]:
plt.scatter(Y_pred, Y_holdout - Y_pred, alpha=0.5)
plt.ylabel("Residual $(y - \hat{y})$")
plt.xlabel("Predicted Prices $(\hat{y})$")
plt.title("Residuals vs Predicted Prices")
plt.title("Residual of prediction for i'th house")
plt.axhline(y = 0, color='r');

#### Question 3.1

As we find from the scatter plot, our model is not perfect. If it were perfect, we would see the identity line (i.e. a line of slope 1). Compute the root mean squared error (RMSE) of the predicted responses: 

$$
\textbf{RMSE} = \sqrt{\frac{1}{n}\sum_{i=1}^n \left( y_i - \hat{y}_i \right)^2 }
$$

Fill out the function below and compute the RMSE for our predictions on both the training data `X_train` and the held out set `X_holdout`.  Your implementation **should not** use for loops.

<!--
BEGIN QUESTION
name: q3
-->

In [ ]:
def rmse(actual_y, predicted_y):
    """
    Args:
        predicted_y: an array of the prediction from the model
        actual_y: an array of the groudtruth label
        
    Returns:
        The root mean square error between the prediction and the groudtruth
    """
    return np.sqrt(np.mean((actual_y-predicted_y)**2))


train_error = rmse(Y_train, linear_model.predict(X_train))
holdout_error = rmse(Y_holdout, linear_model.predict(X_Holdout))


print("Training RMSE:", train_error)
print("Holdout RMSE:", holdout_error)

#### Question 3.2

Is your training error lower than the error on the data the model never got to see? If so, why could this be happening? Answer in the cell below.

**Your response**: Our training error is lower than the error than on the data set that model never got to see. This could be because of overfitting on our X-set so we are actually predicting noise instead of a real model. This makes our predictons on the holdouts to be larger than the training error. 

## Overfitting

Sometimes we can get even lower loss by adding more features. For example, the code below adds the square, square root, and hyperbolic tangent of every feature to the design matrix. We've chosen these bizarre features specifically to highlight overfitting.

In [ ]:
boston_with_extra_features = boston.copy()
for feature_name in boston.columns:
    boston_with_extra_features[feature_name + "^2"] = boston_with_extra_features[feature_name] ** 2
    boston_with_extra_features["sqrt" + feature_name] = np.sqrt(boston_with_extra_features[feature_name])
    boston_with_extra_features["tanh" + feature_name] = np.tanh(boston_with_extra_features[feature_name])  

boston_with_extra_features.head(5)

We split up our data again and refit the model. From this cell forward, we append `2` to the variable names `X_train, X_holdout, Y_train, Y_holdout, train_error, holdout_error` in order to maintain our original data. **Make sure you use these variable names from this cell forward**, at least until we get to the part where we create version 3 of each of these.

In [ ]:
np.random.seed(25)
X = boston_with_extra_features
X_train2, X_holdout2, Y_train2, Y_holdout2 = train_test_split(X, Y, test_size = 0.10)
linear_model.fit(X_train2, Y_train2);

Looking at our training and test RMSE, we see that they are lower than you computed earlier. This strange model is seemingly better, even though it includes seemingly useless features like the hyperbolic tangent of the average number of rooms per dwelling.

In [ ]:
train_error2 = rmse(Y_train2, linear_model.predict(X_train2)) 
holdout_error2 = rmse(Y_holdout2, linear_model.predict(X_holdout2))

print("Training RMSE:", train_error2)
print("Holdout RMSE:", holdout_error2)

The code below generates the training and holdout RMSE for 49 different models stores the results in a DataFrame. The first model uses only the first feature "CRIM". The second model uses the first two features "CRIM" and "ZN", and so forth.

In [ ]:
errors_vs_N = pd.DataFrame(columns = ["N", "Training Error", "Holdout Error"])
range_of_num_features = range(1, X_train2.shape[1] + 1)

for N in range_of_num_features:
    X_train_first_N_features = X_train2.iloc[:, :N]    
    
    linear_model.fit(X_train_first_N_features, Y_train2)
    train_error_overfit = rmse(Y_train2, linear_model.predict(X_train_first_N_features))
    
    X_holdout_first_N_features = X_holdout2.iloc[:, :N]
    holdout_error_overfit = rmse(Y_holdout2, linear_model.predict(X_holdout_first_N_features))    
    errors_vs_N.loc[len(errors_vs_N)] = [N, train_error_overfit, holdout_error_overfit]
    
errors_vs_N.head()

If we plot the training and holdout error as we add each additional feature, our training error gets lower and lower, and in fact it's possible to prove with linear algebra that the training error will decrease monotonically.

By contrast, the error on unseen held out data is higher for the models with more parameters, since the lessons learned from these last 20+ features aren't actually useful when applied to unseen data. That is, these models aren't generalizable.

In [ ]:
sns.lineplot(x = errors_vs_N["N"], y = errors_vs_N["Training Error"], label = "Training Error")
sns.lineplot(x = errors_vs_N["N"], y = errors_vs_N["Holdout Error"], label = "Holdout Error")

This plot is a useful tool for **model selection**: the best model is the one the lowest error on the holdout set, i.e. the one that includes parameters 1 through 28.

## Regularization

As an alternative example, instead of using only the first N features, we can use various different regularization strengths. For example, for really low regularization strengths (e.g. $\alpha = 10^{-12}$), we get a model that is nearly identical to our linear regression model.

In [ ]:
from sklearn.linear_model import Ridge

regularized_model = Ridge(alpha = 1e-12)
regularized_model.fit(X_train2, Y_train2)
ridge_coefs_low_regularization = regularized_model.coef_

linear_model = linear_model.fit(X_train2, Y_train2)
linear_coefs = linear_model.coef_

# what is the difference between the Ridge and OLS coefficients?
(ridge_coefs_low_regularization - linear_coefs).round()

However, if we pick a large regularization strength, e.g. $\alpha = 10^4$, we see that the resulting parameters are much smaller in magnitude. 

In [ ]:
regularized_model = Ridge(alpha = 10**4)
regularized_model.fit(X_train2, Y_train2)
regularized_model.coef_

In [ ]:
Y_train2

### Standard Scaling

In order to properly regularize a model, the features should be at the same scale. Otherwise the model has to spend more of its parameter budget to use "small" features (e.g. lengths in inches) compared to "large" features (e.g. lengths in kilometers).

To do this we can use a Standard Scaler to create a new version of the DataFrame where every column has zero mean and a standard deviation of 1.

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(boston_with_extra_features)
boston_with_extra_features_scaled = pd.DataFrame(ss.transform(boston_with_extra_features), columns = boston_with_extra_features.columns)
boston_with_extra_features_scaled

Let's now regenerate the training and holdout sets using this new rescaled dataset.

In [ ]:
np.random.seed(25)
X = boston_with_extra_features_scaled
X_train3, X_holdout3, Y_train3, Y_holdout3 = train_test_split(X, Y, test_size = 0.10)

Fitting our regularized model with $\alpha = 10^2$ on this scaled data, we now see that our coefficients are of about the same magnitude. This is because all of our features are of around the same magnitude, whereas in the unscaled data, some of the features like TAX^2 were much larger than others.

In [ ]:
from sklearn.linear_model import Ridge
regularized_model = Ridge(alpha = 10**2)
regularized_model.fit(X_train3, Y_train3)
regularized_model.coef_

### Finding an Optimum Alpha

In the cell below, write code that generates a DataFrame with the training and holdout error for the range of alphas given. Make sure you're using the 3rd training and holdout sets, which have been rescaled!

**Note: You should use all features for every single model that you fit, i.e. you're not going to be keeping only the first N features.**

*Hint*: It is possible to "append" or add a row to a DataFrame by calling `loc`, e.g. `df.loc[len(df)] = [2, 3, 4]` (assuming `df` has 3 columns!).

In [ ]:
from sklearn.linear_model import Ridge
error_vs_alpha = pd.DataFrame(columns = ["alpha", "Training Error", "Holdout Error"])
range_of_alphas = 10**np.linspace(-5, 4, 40)

for alpha in range_of_alphas:
    regularized_model = Ridge(alpha= alpha)
    regularized_model.fit(X_train3, Y_train3)

    train_error_overfit = rmse(Y_train3, regularized_model.predict(X_train3))
    holdout_error_overfit = rmse(Y_holdout3, regularized_model.predict(X_holdout3))
    
    error_vs_alpha.loc[len(error_vs_alpha)] = [alpha, train_error_overfit, holdout_error_overfit]

error_vs_alpha

Below we plot your training and holdout set error for the range of alphas given. You should see a figure where training error goes down as model complexity increases, but the error on the held out set is large for extreme values of alpha, and minimized for some intermediate value.

Note that on your plot, the **x-axis is in the inverse of complexity**! In other words, small alpha models (on the left) are complex, because there is no regularization. That's why the training error is lowest on the left side of the plot, as this is where overfitting occurs.

In [ ]:
sns.lineplot(x = error_vs_alpha["alpha"], y = error_vs_alpha["Training Error"], label = "Training Error")
sns.lineplot(x = error_vs_alpha["alpha"], y = error_vs_alpha["Holdout Error"], label = "Holdout Error")

#### Question 4

From the plot above, what is the best alpha to use?

**Your response**: From the plot above it seems pretty clear that we want to use a pretty low alpha where there's a dip in the graph close to 0. In this case (after looking at the table), the best alpha for this is around .41. 

## Test Set vs. Validation Set (a.k.a. Development Set)

In the plots above, we trained our models on a training set, and plotted the resulting RMSE on the training set in blue. We also held out a set of data, and plotted the error on this holdout set in red, calling it the "holdout set error". 

For the example above, since we used the holdout set to pick a hyperparameter, we'd call the holdout set a "validation set" or "development set". These terms are exactly synonomous.

It would not be accurate to call this line the "test set error", because we did not use this dataset as a test set. While it is true that your code never supplied X_test3 or Y_test3 to the fit function of the ridge regression models, once you decide to use the holdout set to select between different models, different hyperparameters, or different sets of features, then we are not using that dataset as a "test set".

That is, since we've used this holdout set for picking alpha, the resulting errors are no longer unbiased predictors of our performance on unseen models -- the true error on an unseen dataset is likely to be somewhat higher than the validation set. After all, we trained 40 models and picked the best one!

In many real world contexts, model builders will split their data into three sets: training, validation, and test sets, where the test set is only ever used once. That is, there are two holdout sets: One used as a development set (for model selection), and one used a test set (for providing an unbiased estimate of error).

## An Alternate Strategy for Hyper Parameter Selection: K-Fold Cross Validation

Earlier we used the holdout method for model selection (the holdout method is also sometimes called "simple cross validation"). Another approach is K-fold cross validation. This allows us to use more data for training instead of having to set aside some specifically for hyperparameter selection. However, doing so requires more computation resources as we'll have to fit K models per hyperparameter choice.

In this class, there's really no reason not to use cross validation. However, in environments where models are very expensive to train (e.g. deep learning), you'll typically prefer using a holdout set (simple cross validation) rather than K-fold cross validation.

To emphasize what K-fold cross validation actually means, we're going to manually carry out the procedure. Recall the approach looks something like the figure below for 5-fold cross validation:

<img src="grid_search_cross_validation.png" width=500px>

When we use K-fold cross validation, rather than using a held out set for model selection, we instead use the training set for model selection. To select between various features, various models, or various hyperparameters, we split the training set further into multiple temporary train and validation sets (each split is called a "fold", hence k-fold cross validation). We will use the average validation error across all k folds to make our optimal feature, model, and hyperparameter choices. In this example, we'll only use this procedure for hyperparameter selection, specifically to choose the best alpha.

#### Question 5

Scikit-learn has built-in support for cross validation.  However, to better understand how cross validation works complete the following function which cross validates a given model. Use X_train3 and Y_train3.

1. Use the [`KFold.split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) function to get 4 splits on the training data. Note that `split` returns the row indices of the data for that split.
2. For **each** split:
    1. Select out the training and validation rows based on the split indices and features.
    2. Compute the RMSE on the validation split.
    3. Return the average error across all cross validation splits.

In [ ]:
X_train3

In [ ]:
from sklearn.model_selection import KFold

def compute_CV_error(model, X_train, Y_train):
    '''
    Split the training data into 4 subsets.
    For each subset, 
        fit a model holding out that subset
        compute the MSE on that subset (the validation set)
    You should be fitting 4 models total.
    Return the average MSE of these 4 folds.

    Args:
        model: an sklearn model with fit and predict functions 
        X_train (data_frame): Training data
        Y_train (data_frame): Label 

    Return:
        the average validation MSE for the 4 splits.
    '''
    kf = KFold(n_splits=4)
    validation_errors = np.empty(0)
    
    for train_idx, valid_idx in kf.split(X_train):
        # split the data
        split_X_train, split_X_valid = X_train.iloc[train_idx,:], X_train.iloc[valid_idx,:]
        split_Y_train, split_Y_valid = Y_train[train_idx], Y_train[valid_idx]

        # Fit the model on the training spli
        model.fit(split_X_train, split_Y_train)

        # Predict housing prices on the test set
        y_pred = model.predict(split_X_valid)
        
        # Compute the RMSE on the validation split
        R_mse = rmse(split_Y_valid, y_pred)

        validation_errors = np.append(validation_errors, R_mse)
        
    return np.mean(validation_errors)

### Question 5

Use `compute_CV_error` to add a new column to `error_vs_alpha` which gives the 4-fold cross validation error for the given choice of alpha.
<!--
BEGIN QUESTION
name: q5
-->

In [ ]:
cv_errors = np.empty(0)
range_of_alphas = 10**np.linspace(-5, 4, 40)

for alpha in range_of_alphas:
    
    cv_errors = np.append(cv_errors,compute_CV_error(Ridge(alpha= alpha), X_train3, Y_train3))
    
error_vs_alpha["CV Error"] = cv_errors
error_vs_alpha

The code below shows the holdout error that we computed in the previous problem as well as the 4-fold cross validation error. Note that the cross validation error shows a similar dependency on alpha relative to the holdout error. This is because they are both doing the same thing, namely trying to estimate the expected error on unseen data drawn from distribution from which the training set was drawn. 

In other words, this figure compares the holdout method with 4-fold cross validation.

In [ ]:
sns.lineplot(x = np.log(error_vs_alpha["alpha"]), y = error_vs_alpha["Holdout Error"], label = "Holdout Error")
sns.lineplot(x = np.log(error_vs_alpha["alpha"]), y = error_vs_alpha["CV Error"], label = "CV Error")

## Submission

Congratulations! You are finished with this assignment.

To double-check your work, the cell below will rerun all of the autograder tests.

## Submission

Make sure you have run all cells in your notebook in order. Then execute the following commands in the File menu:

* Save and Checkpoint
* Close and Halt

Then submit your notebook for Canvas Assignment Lab7.